In [1]:
%load_ext rustdef

load rustdef


<IPython.core.display.Javascript object>

In [2]:
%rustdef deps add --help

usage: %rustdef deps add [-h] args [args ...]

Add dependencies

positional arguments:
  args        Dependencies to be added. See cargo-edit

optional arguments:
  -h, --help  show this help message and exit


In [3]:
# numpy requires ndarray < 0.15.0
%rustdef deps add ndarray@0.14.0 numpy

In [4]:
%rustdef deps show

ndarray = "0.14.0"
numpy = "0.13.1"

[pyo3]
version = "0.13.2"
features = [ "extension-module",]



In [5]:
%%rustdef
use numpy::{IntoPyArray, PyArrayDyn, PyReadonlyArrayDyn};
use ndarray::{ArrayD, ArrayViewD};

// immutable example
fn axpy_rs(a: f64, x: ArrayViewD<'_, f64>, y: ArrayViewD<'_, f64>) -> ArrayD<f64> {
    a * &x + &y
}

#[pyfn(m, "axpy")]
fn axpy<'py>(
    py: Python<'py>,
    a: f64,
    x: PyReadonlyArrayDyn<'py, f64>,
    y: PyReadonlyArrayDyn<'py, f64>,
) -> &'py PyArrayDyn<f64> {
    let x = x.as_array();
    let y = y.as_array();
    axpy_rs(a, x, y).into_pyarray(py)
}

Use previous build


In [6]:
import numpy as np

In [7]:
axpy(2.4, np.array([1.0, 3.2, 2.9]), np.array([2.3, 10.9, -2]))

array([ 4.7 , 18.58,  4.96])

In [8]:
%%rustdef
use numpy::{IntoPyArray, PyArrayDyn, PyReadonlyArrayDyn};
use ndarray::{ArrayD, ArrayViewD};

// mutable example
#[pyfn(m, "mult_inline")]
fn mult_inline<'py>(_py: Python<'py>, a: f64, x: &'py PyArrayDyn<f64>) -> PyResult<()> {
    let mut x = unsafe { x.as_array_mut() };
    x *= a;
    Ok(())
}

Use previous build


In [9]:
x = np.random.rand(10)
y = np.random.rand(10)
x, y

(array([0.17041239, 0.16621988, 0.54191874, 0.9677506 , 0.76994787,
        0.05423104, 0.86507003, 0.27770545, 0.70580458, 0.67681182]),
 array([0.38020526, 0.19851507, 0.09754443, 0.44738533, 0.25492649,
        0.59898851, 0.15901704, 0.9779843 , 0.1517463 , 0.0469063 ]))

In [10]:
axpy(3.0, x, y)

array([0.89144243, 0.69717472, 1.72330065, 3.35063713, 2.56477009,
       0.76168164, 2.75422714, 1.81110064, 2.26916005, 2.07734175])

In [11]:
mult_inline(12.0, x)

In [12]:
x

array([ 2.04494867,  1.99463858,  6.50302488, 11.61300722,  9.2393744 ,
        0.6507725 , 10.38084037,  3.33246535,  8.46965501,  8.12174178])